### Data Preparation

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

import pandas as pd
import re
# import geopandas as gpd

In [64]:
co2_emitters = pd.read_excel('data/epa_ghrp_emitters.xls', skiprows=6, header=0)
# co2_emitters_gdf = gpd.GeoDataFrame(emitters, 
#                                   geometry=gpd.points_from_xy(emitters['LONGITUDE'], emitters['LATITUDE']),
#                                   crs='EPSG:4326')

electronics_co_buyers = pd.read_excel('data/epa_ghrp_co_buyers_electronics_manufacturing.xls', skiprows=6, header=0)
electronics_co_buyers['SECTOR'] = 'Petrochemical Production' 
petrochem_co_buyers = pd.read_excel('data/epa_ghrp_co_buyers_petrochem_prod.xls', skiprows=6, header=0)
petrochem_co_buyers['SECTOR'] = 'Electronics Manufacturing' 

co_buyers = pd.concat([electronics_co_buyers, petrochem_co_buyers], ignore_index=True)
# co_buyers_gdf = gpd.GeoDataFrame(co_buyers, 
#                                   geometry=gpd.points_from_xy(co_buyers['LONGITUDE'], co_buyers['LATITUDE']),
#                                   crs='EPSG:4326')

def clean_column_names(df):
    df.columns = [re.sub(r'[\s()]', '_', c.upper()).replace("__", "_").replace("/", "_PER_").rstrip("_") for c in df.columns]

for df in [co2_emitters, co_buyers]:
    clean_column_names(df)

### Collecitng the best primary emitter candidates

In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.neighbors import BallTree
from tqdm import tqdm

EARTH_RADIUS_KM = 6371.0088

def _to_radians(df):
    return np.deg2rad(df[["LATITUDE", "LONGITUDE"]].values)  # BallTree expects [lat, lon] in radians

def build_buyer_seller_graph_km(df_buyers, df_sellers, L_km=50.0):
    """
    Build a bipartite graph with edges between buyers and sellers within L_km (great-circle distance).
    Edge weights are stored in kilometers.
    """
    # Basic sanity: drop rows with missing coords
    buyers = df_buyers.dropna(subset=["LONGITUDE", "LATITUDE"]).copy()
    sellers = df_sellers.dropna(subset=["LONGITUDE", "LATITUDE"]).copy()

    # Build BallTree on sellers (haversine metric)
    sellers_rad = _to_radians(sellers)
    tree = BallTree(sellers_rad, metric="haversine")

    # Query buyers against sellers within radius (in radians)
    buyers_rad = _to_radians(buyers)
    r_rad = L_km / EARTH_RADIUS_KM
    ind_array, dist_array = tree.query_radius(buyers_rad, r=r_rad, return_distance=True, sort_results=True)

    # Graph
    G = nx.Graph()

    # Add sellers
    for _, row in sellers.iterrows():
        G.add_node(f"S_{row['GHGRP_ID']}",
                   bipartite="seller",
                   pos=(row["LONGITUDE"], row["LATITUDE"]))

    # Add buyers
    for _, row in buyers.iterrows():
        G.add_node(f"B_{row['GHGRP_ID']}",
                   bipartite="buyer",
                   pos=(row["LONGITUDE"], row["LATITUDE"]))

    # Add edges (distance in km)
    for b_idx, (seller_idxs, d_rads) in tqdm(enumerate(zip(ind_array, dist_array))):
        buyer_id = f"B_{buyers.iloc[b_idx]['GHGRP_ID']}"
        for s_idx, d_rad in zip(seller_idxs, d_rads):
            seller_id = f"S_{sellers.iloc[s_idx]['GHGRP_ID']}"
            d_km = float(d_rad * EARTH_RADIUS_KM)
            G.add_edge(buyer_id, seller_id, weight=d_km)

    return G

def plot_graph(G, title="Buyer–Seller Graph (great-circle distances in km)"):
    pos = nx.get_node_attributes(G, "pos")
    node_colors = ["lightgreen" if G.nodes[n].get("bipartite")=="buyer" else "skyblue" for n in G.nodes]

    plt.figure(figsize=(9, 7))
    nx.draw(
        G, pos,
        node_size=100,
        node_color=node_colors,
        edge_color="gray",
        with_labels=True,
        font_size=7
    )
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(title)
    plt.show()


def filter_sellers_km(G, df_sellers, x, j, L_km):
    """
    Return sellers with GHG QUANTITY >= x and at least j buyers with edge distance <= L_km (km).
    Assumes edge 'weight' is in km as built above.
    """
    # Map GHGRP_ID -> GHG QUANTITY
    seller_info = df_sellers.set_index("GHGRP_ID")["GHG_QUANTITY_METRIC_TONS_CO2E"].to_dict()
    qualified = []

    for node in G.nodes:
        if G.nodes[node].get("bipartite") != "seller":
            continue

        ghgrp_id = int(node.split("_", 1)[1])
        ghg_qty = seller_info.get(ghgrp_id, None)
        if ghg_qty is None or ghg_qty < x:
            continue

        # Collect buyers within L_km along with distances
        buyer_neighbors_with_d = [
            (nbr, data.get("weight", np.inf))
            for nbr, data in G[node].items()
            if G.nodes[nbr].get("bipartite") == "buyer" and data.get("weight", np.inf) <= L_km
        ]

        if len(buyer_neighbors_with_d) >= j:
            # Sort buyers by distance once
            buyer_neighbors_with_d.sort(key=lambda t: t[1])

            # Extract aligned lists
            buyer_ids_sorted = [int(nbr.split("_", 1)[1]) for nbr, _d in buyer_neighbors_with_d]
            distances_sorted = [round(float(d_km), 3) for _nbr, d_km in buyer_neighbors_with_d]

            buyers_csv = ",".join(map(str, buyer_ids_sorted))
            distances_csv = ",".join(map(str, distances_sorted))

            qualified.append({
                "CO2_EMITTER_ID": ghgrp_id,
                "GHG_QUANTITY": ghg_qty,
                f"BUYER_COUNT_WITHIN_{L_km}KM": len(buyer_neighbors_with_d),
                f"BUYER_GHGRP_IDS_WITHIN_{L_km}KM": buyers_csv,
                f"BUYER_DISTANCES_WITHIN_{L_km}KM": distances_csv
            })

    return pd.DataFrame(qualified)

In [106]:
print(co_buyers.shape, co2_emitters.shape)
G = build_buyer_seller_graph_km(co_buyers, co2_emitters, L_km=100)
candidates_base = filter_sellers_km(
    G, 
    co2_emitters, 
    5000, # minimum emissions quantity
    2, # minimum buyers available, 
    50 # maximum co_buyer distance (km)
)
candidates_base.head()

(121, 14) (7511, 13)


121it [00:00, 142.99it/s]


,CO2_EMITTER_ID,GHG_QUANTITY,BUYER_COUNT_WITHIN_50KM,BUYER_GHGRP_IDS_WITHIN_50KM,BUYER_DISTANCES_WITHIN_50KM
0,1004377,288302,4,"1010134,1008059,1011813,1003945","33.639,37.416,37.904,45.42"
1,1012507,286498,9,"1007002,1001711,1002859,1001713,1002758,100665...","23.497,24.818,25.38,28.51,29.195,31.341,32.968..."
2,1009170,13382,2,"1007002,1002859","48.945,49.823"
3,1012472,92930,9,"1007002,1001711,1002859,1001713,1002758,100665...","23.497,24.818,25.38,28.51,29.195,31.341,32.968..."
4,1012466,89986,9,"1007002,1001711,1002859,1001713,1002758,100665...","23.497,24.818,25.38,28.51,29.195,31.341,32.968..."


### Feature Engineering

In [111]:
# Emittor Dataset Attribution
candidates = candidates_base.merge(
    co2_emitters.drop(["GHG_QUANTITY_METRIC_TONS_CO2E"], axis=1), 
    left_on="CO2_EMITTER_ID", 
    right_on="GHGRP_ID", 
    how="left"
).drop(["GHGRP_ID"], axis=1)

# Turnover Labs Conversion Factors
candidates.rename(columns={"GHG_QUANTITY": "CO2_EMISSIONS_MT"}, inplace=True)
candidates.insert(2, "TL_COPROD_MT", candidates["CO2_EMISSIONS_MT"] * (63 / 100))
candidates.insert(3, "TL_COPROD_ELECTRUSAGE_FULLUTIL_KWH", candidates["TL_COPROD_MT"]*4830)
candidates.insert(4, "TL_CH30HPROD_MT", candidates["CO2_EMISSIONS_MT"] * (72 / 100))
candidates.insert(5, "TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_KWH", candidates["TL_CH30HPROD_MT"]*5320)
candidates.head()

,CO2_EMITTER_ID,CO2_EMISSIONS_MT,TL_COPROD_MT,TL_COPROD_ELECTRUSAGE_FULLUTIL_KWH,TL_CH30HPROD_MT,TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_KWH,BUYER_COUNT_WITHIN_50KM,BUYER_GHGRP_IDS_WITHIN_50KM,BUYER_DISTANCES_WITHIN_50KM,REPORTING_YEAR,FACILITY_NAME,REPORTED_ADDRESS,LATITUDE,LONGITUDE,CITY_NAME,COUNTY_NAME,STATE,ZIP_CODE,PARENT_COMPANIES,SUBPARTS
0,1004377,288302,181630.26,877274155.8,207577.44,1.104312e+09,4,"1010134,1008059,1011813,1003945","33.639,37.416,37.904,45.42",2023,121 REGIONAL DISPOSAL FACILITY,3820 SAM RAYBURN HIGHWAY,33.29857,-96.535860,MELISSA,COLLIN COUNTY,TX,75454,NORTH TEXAS MUNICIPAL WATER DISTRICT (100%),HH
1,1012507,286498,180493.74,871784764.2,206278.56,1.097402e+09,9,"1007002,1001711,1002859,1001713,1002758,100665...","23.497,24.818,25.38,28.51,29.195,31.341,32.968...",2023,220 Gulf Coast Basin,"811 Louisiana, Suite 2100",29.75993,-95.366413,Houston,HARRIS COUNTY,TX,77002,TARGA RESOURCES CORP (100%),W
2,1009170,13382,8430.66,40720087.8,9635.04,5.125841e+07,2,"1007002,1002859","48.945,49.823",2023,260 East Texas Basin - BP America Production C...,501 Westlake Park Blvd.,29.78037,-95.629500,Houston,NaN,TX,77079,BP AMERICA INC (100%),W
3,1012472,92930,58545.90,282776697.0,66909.60,3.559591e+08,9,"1007002,1001711,1002859,1001713,1002758,100665...","23.497,24.818,25.38,28.51,29.195,31.341,32.968...",2023,345 Arkoma Basin,"811 Louisiana, Suite 2100",29.75993,-95.366413,Houston,HARRIS COUNTY,TX,77002,TARGA RESOURCES CORP (100%),W
4,1012466,89986,56691.18,273818399.4,64789.92,3.446824e+08,9,"1007002,1001711,1002859,1001713,1002758,100665...","23.497,24.818,25.38,28.51,29.195,31.341,32.968...",2023,350 South Oklahoma Folded Belt,"811 Louisiana, Suite 2100",29.75993,-95.366413,Houston,HARRIS COUNTY,TX,77002,TARGA RESOURCES CORP (100%),W


In [112]:
power_price_df = pd.read_excel('data/monthly_power_price_by_state_eia_may_2025.xls')
# https://www.eia.gov/electricity/monthly/epm_table_grapher.php?t=epmt_5_6_a
clean_column_names(power_price_df)

us_states_to_acronyms = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC', 'American Samoa': 'AS', 'Guam': 'GU',
    'Northern Mariana Islands': 'MP', 'Puerto Rico': 'PR', 'U.S. Virgin Islands': 'VI'
}
power_price_df['STATE_ABBR'] = power_price_df['CENSUS_DIVISION_OR_STATE'].map(us_states_to_acronyms)
state_to_power_price_dict = dict(zip(power_price_df['STATE_ABBR'], power_price_df['INDUSTRIAL_MAY_25']))

candidates.insert(4, "TL_COPROD_ELECTRUSAGE_FULLUTIL_COST_USD", 
                  [state_to_power_price_dict[state] * kwh_usage / 100 for state, kwh_usage in zip(candidates['STATE'], candidates['TL_COPROD_ELECTRUSAGE_FULLUTIL_KWH'])]
)
candidates.insert(7, "TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_COST_USD", 
                  [state_to_power_price_dict[state] * kwh_usage / 100 for state, kwh_usage in zip(candidates['STATE'], candidates['TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_KWH'])]
)
candidates.head()

,CO2_EMITTER_ID,CO2_EMISSIONS_MT,TL_COPROD_MT,TL_COPROD_ELECTRUSAGE_FULLUTIL_KWH,TL_COPROD_ELECTRUSAGE_FULLUTIL_COST_USD,TL_CH30HPROD_MT,TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_KWH,TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_COST_USD,BUYER_COUNT_WITHIN_50KM,BUYER_GHGRP_IDS_WITHIN_50KM,...,FACILITY_NAME,REPORTED_ADDRESS,LATITUDE,LONGITUDE,CITY_NAME,COUNTY_NAME,STATE,ZIP_CODE,PARENT_COMPANIES,SUBPARTS
0,1004377,288302,181630.26,877274155.8,5.816328e+07,207577.44,1.104312e+09,7.321588e+07,4,"1010134,1008059,1011813,1003945",...,121 REGIONAL DISPOSAL FACILITY,3820 SAM RAYBURN HIGHWAY,33.29857,-96.535860,MELISSA,COLLIN COUNTY,TX,75454,NORTH TEXAS MUNICIPAL WATER DISTRICT (100%),HH
1,1012507,286498,180493.74,871784764.2,5.779933e+07,206278.56,1.097402e+09,7.275775e+07,9,"1007002,1001711,1002859,1001713,1002758,100665...",...,220 Gulf Coast Basin,"811 Louisiana, Suite 2100",29.75993,-95.366413,Houston,HARRIS COUNTY,TX,77002,TARGA RESOURCES CORP (100%),W
2,1009170,13382,8430.66,40720087.8,2.699742e+06,9635.04,5.125841e+07,3.398433e+06,2,"1007002,1002859",...,260 East Texas Basin - BP America Production C...,501 Westlake Park Blvd.,29.78037,-95.629500,Houston,NaN,TX,77079,BP AMERICA INC (100%),W
3,1012472,92930,58545.90,282776697.0,1.874810e+07,66909.60,3.559591e+08,2.360009e+07,9,"1007002,1001711,1002859,1001713,1002758,100665...",...,345 Arkoma Basin,"811 Louisiana, Suite 2100",29.75993,-95.366413,Houston,HARRIS COUNTY,TX,77002,TARGA RESOURCES CORP (100%),W
4,1012466,89986,56691.18,273818399.4,1.815416e+07,64789.92,3.446824e+08,2.285244e+07,9,"1007002,1001711,1002859,1001713,1002758,100665...",...,350 South Oklahoma Folded Belt,"811 Louisiana, Suite 2100",29.75993,-95.366413,Houston,HARRIS COUNTY,TX,77002,TARGA RESOURCES CORP (100%),W


In [ ]:
# This will also differentiate based on the buyer, region, sector, etc.
CO_MT_PRICE_USD = 200 # Turnover Labs internal estimate seems to actually be 600
CH30H_MT_PRICE_USD = 778 # based on methanold sold by Methanex https://www.methanex.com/our-products/about-methanol/pricing/

candidates.insert(6, "TL_COPROD_FULLUTIL_REVENUE_USD", candidates["TL_COPROD_MT"] * CO_MT_PRICE_USD)
candidates.insert(7, "TL_COPROD_FULLUTIL_PROFIT_MARGIN_USD", candidates["TL_COPROD_FULLUTIL_REVENUE_USD"] - candidates["TL_COPROD_ELECTRUSAGE_FULLUTIL_COST_USD"])
candidates.insert(11, "TL_CH30HPROD_FULLUTIL_REVENUE_USD", candidates["TL_CH30HPROD_MT"] * CH30H_MT_PRICE_USD)
candidates.insert(12, "TL_CH30HPROD_FULLUTIL_PROFIT_MARGIN_USD", candidates["TL_CH30HPROD_FULLUTIL_REVENUE_USD"] - candidates["TL_CH30HPROD_ELECTRUSAGE_FULLUTIL_COST_USD"])

### Proximity Competition: Methanol Suppliers